In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import chardet

file_path = r"D:\pythondata\★_재배치_회수.csv"

# 인코딩 감지
with open(file_path, 'rb') as f:
    rawdata = f.read(10000)
    detected_encoding = chardet.detect(rawdata)['encoding']
    print(f"✅ 감지된 인코딩: {detected_encoding}")

# 파일 처음 몇 줄 미리 보기 (정확한 구분자 파악)
with open(file_path, 'r', encoding=detected_encoding, errors='replace') as f:
    for i in range(5):
        print(f.readline())

✅ 감지된 인코딩: UTF-8-SIG
자전거번호	회수대여소	회수대여소명	회수일시	회수시간

SPB-42889	102	 망원역 1번출구 앞	2023-01-01	4:41:10 PM

SPB-41193	102	 망원역 1번출구 앞	2023-01-01	4:43:04 PM

SPB-80219	102	 망원역 1번출구 앞	2023-01-01	8:21:17 PM

SPB-30519	102	 망원역 1번출구 앞	2023-01-01	8:22:03 PM



In [3]:
# 파일명	내용	포맷
# 방문코드_전체데이터.csv	전체 재배치 기록 + 방문코드	CSV (제한 없음)
# 방문코드_요약.xlsx	방문코드별 레코드 수 요약	Excel (엑셀 사용 편리)

# 요청 항목	반영 여부	설명
# 날짜+시간 컬럼 추가	✅	회수일시 + 회수시간 결합
# 오름차순 정렬 기준 변경	✅	"회수대여소" → "날짜+시간" 순서
# 기존 분석/저장 구조 유지	✅	방문코드, 순번, 자전거대수, 요약 저장

# 내용	처리 방식
# NaN 회수대여소	방문코드 계산에서 제외
# A0001 시작	회수대여소 정렬 후, 첫 유효 데이터부터 시작
# 요약	NaN 제외한 방문코드 기준으로 생성
# 최종 저장	정제된 df_valid + NaN 포함 df_null 병합하여 전체 저장

# 항목	결과
# 회수대여소 순서	숫자 기준 오름차순 (102, 105, 999, …)
# 방문코드 A0001	가장 작은 회수대여소에서 시작
# NaN 대여소	맨 뒤로 정렬되며, 방문코드도 이후 번호 부여
# 순번	각 방문코드 내에서 항상 1부터 시작

# 회수대여소	날짜+시간 차이	방문코드 변화	이유
# 102	-	A0001	첫 행
# 102	20분 차이	A0001 유지	같은 대여소, 시간차도 짧음
# 105	5분 차이	A0002 시작	대여소가 달라짐
# 105	35분 차이	A0003 시작	30분 초과

In [4]:
import pandas as pd
import os

# 1. CSV 파일 경로 지정
file_path = r"D:\pythondata\★_재배치_회수.csv"

# 2. 안전하게 불러오기
df = pd.read_csv(
    file_path,
    sep="\t",
    encoding="utf-8-sig",
    dtype={"회수대여소": str}
)

# 3. '날짜+시간' 컬럼 생성
df["날짜+시간"] = pd.to_datetime(df["회수일시"] + " " + df["회수시간"])

# 4. 회수대여소 숫자형 정렬용 컬럼 생성
df["회수대여소_정렬"] = pd.to_numeric(df["회수대여소"], errors="coerce")

# 5. 정렬: 회수대여소 → 날짜+시간
df = df.sort_values(by=["회수대여소_정렬", "날짜+시간"]).reset_index(drop=True)

# 6. 방문코드 생성 (30분 이상 경과 또는 회수대여소 변경 시 새로운 코드)
visit_code = []
visit_index = 1
visit_code.append(f"A{visit_index:04d}")

for i in range(1, len(df)):
    time_diff = df.loc[i, "날짜+시간"] - df.loc[i - 1, "날짜+시간"]
    location_changed = df.loc[i, "회수대여소"] != df.loc[i - 1, "회수대여소"]
    if time_diff.total_seconds() > 1800 or location_changed:
        visit_index += 1
    visit_code.append(f"A{visit_index:04d}")

df["방문코드"] = visit_code

# 7. 순번: 방문코드 기준으로 항상 1부터 시작
df["순번"] = df.groupby("방문코드").cumcount() + 1

# 8. 자전거대수 계산
df["자전거대수"] = df.groupby("방문코드")["회수대여소"].transform("count")

# 9. 요약 테이블
summary = df.groupby("방문코드")["회수대여소"].count().reset_index()
summary.columns = ["방문코드", "레코드수"]

# 10. 정리
df.drop(columns=["회수대여소_정렬"], inplace=True)

# 11. 저장
output_csv_path = r"D:\pythondata\회수_방문코드_전체데이터.csv"
df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

output_summary_path = r"D:\pythondata\회수_방문코드_요약.xlsx"
with pd.ExcelWriter(output_summary_path, engine="openpyxl") as writer:
    summary.to_excel(writer, sheet_name="요약", index=False)

print("✅ CSV 저장 완료:", output_csv_path)
print("✅ 요약 엑셀 저장 완료:", output_summary_path)

✅ CSV 저장 완료: D:\pythondata\회수_방문코드_전체데이터.csv
✅ 요약 엑셀 저장 완료: D:\pythondata\회수_방문코드_요약.xlsx
